# Integrating H2O AutoML and Google BigQuery

Today, I developed a simple proof of concept on how Google BigQuery and H2O AutoML can work together. BigQuery is a highly scalable, serverless data warehouse built on top of Google’s infrastructure. Using the BigQuery console or API, users can query large batches of data from their warehouse very quickly, using Standard SQL. SQL is good for some simple analysis, but often, just using SQL queries leaves much to be desired. This is where H2O AutoML helps. With some simple data transformations using Python, data queried from BigQuery can be passed to H2O AutoML, an automated machine learning workflow that performs many of the modeling tasks typically require multiple lines of code. 

I wrote a simple class that will allow a user to write a query to the BigQuery project of their choosing and then call H2O AutoML to work on the data and return predictions new data as provided. These predictions can be written back to Google BigQuery as a new table. 

## Steps Taken:

Create instance of class GoogleH2OIntegration with desired specifications

In [1]:
from h2o_and_google_bigquery import GoogleH2OIntegration

dataset = 'iris_dataset'
pred_table = 'prediction_table'
bq_auth = '/Users/npng/Downloads/h2o-project-090347f40536.json'

g_h2o = GoogleH2OIntegration(dataset, pred_table, bq_auth=bq_auth)

Call method .bigquery_query() to send a query to Google BigQuery

In [2]:
df = g_h2o.bigquery_query()

Enter/Paste your content. 'end_query' to save it.
SELECT * FROM iris_dataset.iris_table
end_query


g_h2o.bigquery_query() returns a dataframe that can then be manipulated using python. 

In [3]:
df.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,24,5.1,3.3,1.7,0.5,Iris-setosa
1,20,5.1,3.8,1.5,0.3,Iris-setosa
2,19,5.7,3.8,1.7,0.3,Iris-setosa
3,7,4.6,3.4,1.4,0.3,Iris-setosa
4,18,5.1,3.5,1.4,0.3,Iris-setosa


At this point, the user can perform any necessary feature engineering or data exploration in order to generate a finalized dataframe to pass to H2O AutoML. In the case of the dataset being used for this example, no feature engineering is necessary so, we will pass it directly to AutoML after creating separate dataframes for training and validation

In [4]:
from sklearn.model_selection import train_test_split
import numpy as np

y = np.zeros((df.shape[0], 1))

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
ids = df['Id']
df = df.drop(['Id','Species'], axis=1)

In [5]:
print(df.shape, X_train.shape, X_test.shape)

(150, 4) (120, 6) (30, 6)


The finalized dataframes can now be passed to AutoML and allow it to run. Calling the method h2o_automl() with the necessary arguments will perform the complete process of initializing H2O, running AutoML, and returning back the fully trained AutoML object. 

* H2O keyword arguments can be found here: http://docs.h2o.ai/h2o/latest-stable/h2o-docs/starting-h2o.html#from-python
* AutoML keyword arguments can be found here: http://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html

In [6]:
h2o_args = {"nthreads":-1, "max_mem_size":'2g', "ip":"127.0.0.1", "port":54321}
aml_args = {"max_runtime_secs":30}
aml = g_h2o.h2o_automl(X_train, X_test, 'Species', ['Species', 'Id'], h2o_args, aml_args)

Checking whether there is an H2O instance running at http://127.0.0.1:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-macosx) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-macosx) (build 25.121-b15, mixed mode)
  Starting server from /Users/npng/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/xf/ln59kgjs54vbxj19v_3ndpp80000gp/T/tmp4midjd4x
  JVM stdout: /var/folders/xf/ln59kgjs54vbxj19v_3ndpp80000gp/T/tmp4midjd4x/h2o_npng_started_from_python.out
  JVM stderr: /var/folders/xf/ln59kgjs54vbxj19v_3ndpp80000gp/T/tmp4midjd4x/h2o_npng_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster version:,3.16.0.2
H2O cluster version age:,2 months and 8 days
H2O cluster name:,H2O_from_python_npng_qsanow
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


model_id,mean_per_class_error
GBM_grid_0_AutoML_20180208_150703_model_0,0
GLM_grid_0_AutoML_20180208_150703_model_0,0
XRT_0_AutoML_20180208_150703,0
GBM_grid_0_AutoML_20180208_150703_model_1,0
GBM_grid_0_AutoML_20180208_150703_model_24,0.030303
DeepLearning_0_AutoML_20180208_150703,0.030303
GBM_grid_0_AutoML_20180208_150703_model_13,0.030303
GBM_grid_0_AutoML_20180208_150703_model_3,0.030303
DRF_0_AutoML_20180208_150703,0.030303
GBM_grid_0_AutoML_20180208_150703_model_8,0.030303


Using this returned AutoML object, predictions can be made on new data. In this case, I am predicting on the full dataset and writing the results back to a new table in BigQuery, but new data can easily be obtained by calling bigquery_query() one more time.

In [7]:
import h2o
all_data = h2o.H2OFrame.from_python(df, column_names=list(df.columns))
pred = aml.leader.predict(all_data).as_data_frame()
pred

Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


,predict,Iris-setosa,Iris-versicolor,Iris-virginica
0,Iris-setosa,9.999999e-01,5.431858e-08,4.792589e-08
1,Iris-setosa,9.999999e-01,5.431868e-08,4.792577e-08
2,Iris-setosa,9.999999e-01,8.378309e-08,6.467697e-08
3,Iris-setosa,9.999999e-01,5.431864e-08,4.661087e-08
4,Iris-setosa,9.999999e-01,5.431867e-08,4.792577e-08
5,Iris-setosa,9.999999e-01,5.767959e-08,5.004400e-08
6,Iris-setosa,9.999999e-01,5.429483e-08,4.792697e-08
7,Iris-setosa,9.999998e-01,1.666696e-07,5.012870e-08
8,Iris-setosa,9.999999e-01,5.429138e-08,4.792047e-08
9,Iris-setosa,9.999999e-01,5.431865e-08,5.270995e-08


In [8]:
g_h2o.write_to_table(ids.astype(str), pred['predict'])

Success


In [10]:
g_h2o.bigquery_query()

Enter/Paste your content. 'end_query' to save it.
SELECT * FROM iris_dataset.prediction_table
LIMIT 10
end_query


,test_id,prediction
0,5,Iris-setosa
1,3,Iris-setosa
2,115,Iris-virginica
3,149,Iris-virginica
4,130,Iris-virginica
5,75,Iris-versicolor
6,74,Iris-versicolor
7,83,Iris-versicolor
8,81,Iris-versicolor
9,56,Iris-versicolor
